## Quality Checks and their Descriptions
Each of the quality checks has the following structure:
1) Description of what it verifies
2) Data quality dimension
3) Severity level
4) SQL logic and code, which shows how many records did not pass the check

In the following part I load CSV to persistent database file. This way I can use it in multiple SQL queries, therefore for all quality checks.

(The printed number is representing the number of loaded rows from the CSV file.)

In [90]:
import pandas as pd
import sqlite3

df = pd.read_csv('data.csv', encoding='latin1')
conn = sqlite3.connect('temp.db')
df.to_sql('data_table', conn, if_exists='replace', index=False)

541909

### 1. Data Completeness Check
1) Verifies: **whether the crucial fields (invoice number and customer id) are present**
2) Data quality dimension: **Completeness**
3) Severity level: **Critical**
4) SQL logic:

In [79]:
sql_query_1 = """
SELECT 
    COUNT(*) AS failed_records,
    'Critical' AS severity,
    'Missing Invoice Number or CustomerID' AS check_description
FROM data_table
WHERE InvoiceNo IS NULL 
   OR CustomerID IS NULL
"""

check_result_1 = pd.read_sql(sql_query_1, conn)
check_result_1

,failed_records,severity,check_description
0,135080,Critical,Missing Invoice Number or CustomerID


### 2. Data Validity Check
1) Verifies: **whether quantity and price values are positive numbers**
2) Data quality dimension: **Validity**
3) Severity level: **Critical**
4) SQL logic:

In [80]:
sql_query_2 = """
SELECT 
    COUNT(*) AS failed_records,
    'Critical' AS severity,
    'Invalid Quantity or Price Values' AS check_description
FROM data_table
WHERE Quantity <= 0 
   OR UnitPrice <= 0
"""
check_result_2 = pd.read_sql(sql_query_2, conn)
check_result_2

,failed_records,severity,check_description
0,11805,Critical,Invalid Quantity or Price Values


### 3. Data Uniqueness Check
1) Verifies: **whether in each invoice all products are unique**
2) Data quality dimension: **Uniqueness**
3) Severity level: **Critical**
4) SQL logic:

In [82]:
sql_query_3 = """
SELECT 
    COUNT(*) AS failed_records,
    'Critical' AS severity,
    'Duplicate Products in One Invoice' AS check_description
FROM (
    SELECT InvoiceNo, StockCode
    FROM data_table
    GROUP BY InvoiceNo, StockCode
    HAVING COUNT(*) > 1
)
"""
check_result_3 = pd.read_sql(sql_query_3, conn)
check_result_3

,failed_records,severity,check_description
0,9694,Critical,Duplicate Products in One Invoice


### 4. Data Accuracy Check
1) Verifies: **whether place of the order is an existing independent country**
2) Data quality dimension: **Accuracy**
3) Severity level: **Warning**
4) SQL logic:

In [83]:
sql_query_4 = """
SELECT 
    COUNT(*) AS failed_records,
    'Warning' AS severity,
    'Invalid Country Values' AS check_description
FROM data_table
WHERE country NOT IN (
    'United Kingdom', 'Germany', 'France', 'EIRE',
    'Switzerland', 'Spain', 'Portugal', 'Belgium', 
    'Australia', 'Norway', 'Italy', 'RSA',
    'Japan', 'Sweden', 'Astria', 'Denmark', 
    'Poland', 'USA', 'Finland', 'Netherlands',
    'Greece', 'Israel', 'Singapore', 'Canada',
    'Cyprus', 'Lithuania', 'Brazil', 'Czech Republic', 
    'Iceland', 'Malta', 'United Arab Emirates',
    'Lebanon', 'Bahrain', 'Saudi Arabia'
) 
AND country IS NOT NULL 
AND country != '';
"""
check_result_4 = pd.read_sql(sql_query_4, conn)
check_result_4

,failed_records,severity,check_description
0,1954,Warning,Invalid Country Values


### 5. Data Consistency Check
1) Verifies: **whether product's description is consistent**
2) Data quality dimension: **Consistency**
3) Severity level: **Warning**
4) SQL logic:

In [84]:
sql_query_5 = """
SELECT 
    COUNT(*) AS failed_records,
    'Warning' AS severity,
    'Products with Inconsistent Description' AS check_description
FROM (
    SELECT StockCode, COUNT(DISTINCT Description) AS description_count
    FROM data_table
    WHERE StockCode IS NOT NULL
    GROUP BY StockCode
    HAVING description_count > 1
)
"""

check_result_5 = pd.read_sql(sql_query_5, conn)
print(check_result_5)

   failed_records severity                       check_description
0             650  Warning  Products with Inconsistent Description


After detecting records which failed chosen checks, we could get discard them from the table:

In [92]:
sql_delete_1 = """
DELETE FROM data_table
WHERE InvoiceNo IS NULL 
   OR CustomerID IS NULL
"""

sql_delete_2 = """
DELETE FROM data_table
WHERE Quantity <= 0 
   OR UnitPrice <= 0
"""

sql_delete_3 = """
DELETE FROM data_table
WHERE RowID NOT IN (
    SELECT MIN(RowID)
    FROM data_table
    GROUP BY InvoiceNo, StockCode
)
"""

sql_delete_4 = """
DELETE FROM data_table
WHERE country NOT IN (
    'United Kingdom', 'Germany', 'France', 'EIRE',
    'Switzerland', 'Spain', 'Portugal', 'Belgium', 
    'Australia', 'Norway', 'Italy', 'RSA',
    'Japan', 'Sweden', 'Astria', 'Denmark', 
    'Poland', 'USA', 'Finland', 'Netherlands',
    'Greece', 'Israel', 'Singapore', 'Canada',
    'Cyprus', 'Lithuania', 'Brazil', 'Czech Republic', 
    'Iceland', 'Malta', 'United Arab Emirates',
    'Lebanon', 'Bahrain', 'Saudi Arabia'
) 
AND country IS NOT NULL 
AND country != ''
"""

sql_delete_5 = """
DELETE FROM data_table
WHERE StockCode IN (
    SELECT StockCode
    FROM data_table
    WHERE StockCode IS NOT NULL
    GROUP BY StockCode
    HAVING COUNT(DISTINCT Description) > 1
)
"""


cursor = conn.cursor()
deleted_records_number = 0

cursor.execute(sql_delete_1)
deleted_records_number += cursor.rowcount

cursor.execute(sql_delete_2)
deleted_records_number += cursor.rowcount

cursor.execute(sql_delete_3)
deleted_records_number += cursor.rowcount

cursor.execute(sql_delete_4)
deleted_records_number += cursor.rowcount

cursor.execute(sql_delete_5)
deleted_records_number += cursor.rowcount

conn.commit()

print(f"Deleted {deleted_records_number} records")

Deleted 193043 records


Close the connection, since all queries are done:

In [93]:
conn.close()